In [1]:
!pip install evaluate sacrebleu transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [2]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## CHATGPT Evaluation

In [6]:
from evaluate import load
import time
from sacrebleu import sentence_bleu
import sacrebleu

import pandas as pd

start_time = time.time()
perplexity = load("perplexity", module_type="metric")

#USE THIS TO GET SCORES 
 #Perplexity 
 #LM_prob 
 #Pseudo BLEU
 # 
def get_scores_optimized(input_file, output_file): 
  df = pd.read_csv(input_file)
  gens = df["text"].fillna(' ')
  pseudo = df["pseudo"].fillna(' ')

  results = perplexity.compute(model_id='gpt2',
                             predictions=gens )
  df['LM_prob'] = [1/i for i in results['perplexities']]
  df['PPL'] = results['perplexities']
  
  for i, g in enumerate(gens):
    print(g, [pseudo[i]])
    sentence_bleu(g, [pseudo[i]]).score
  
  df['pseudo_bleu'] = [ sentence_bleu(g, [pseudo[i]]).score for i, g in enumerate(gens)]

  df['bbleu'] = [sacrebleu.corpus_bleu(g, [pseudo[i]], smooth_method="exp") for i, g in enumerate(gens)]
  bleurt = sacrebleu.BLEU('BLEURT-extended')
  df['bleurt'] = [bleurt.corpus_score(g, [pseudo[i]]) for i, g in enumerate(gens)]

 
  df.to_csv(output_file)


# print("10")
# get_scores_optimized("/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/tst/10_one/filtered_bleurt/personality-vertical-tst-10-filter.csv", "/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/tst/10_one/filtered_bleurt/personality-vertical-tst-10-filter-lm.csv")
# print("10 not filtered")
# get_scores_optimized("/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/tst/10_one/personality-vertical-tst-10.csv", "/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/tst/10_one/personality-vertical-tst-10-lm.csv")
# print(36)
# get_scores_optimized("/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/tst/36_one/personality-vertical-tst-36.csv", "/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/tst/36_one/personality-vertical-tst-36-lm.csv")
# print(1)
# get_scores_optimized("/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/tst/1_all/personality-vertical-tst-1.csv", "/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/tst/1_all/personality-vertical-tst-1-lm.csv")
#print(6)
#get_scores_optimized("/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/viggo-experiments/viggo-vertical-filterd-10.csv", "/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/viggo-experiments/viggo-vertical-filterd-10-lmbleu.csv")
get_scores_optimized("/content/drive/MyDrive/AA NLP244 Project Experiments/Evaluation/ChatGPT/NLP244_TV_chatgpt_90_responses.csv", "/content/drive/MyDrive/AA NLP244 Project Experiments/Evaluation/ChatGPT/chatgpt_eval.csv")
print("--- %s seconds ---" % (time.time() - start_time))

Using pad_token, but it is not set yet.


  0%|          | 0/6 [00:00<?, ?it/s]

Sasural Simar Ka is a television drama series that falls under the genre of drama." [' Sasural Simar Ka genres drama television series  <|endoftext|>  . Sasural Simar Ka genres drama television series  <|endoftext|>  . Sasural Simar Ka genres drama television series  <|endoftext|>  . Sasural Simar Ka genres drama television series  <|endoftext|>  . Sasural Simar Ka genres drama television series  <|endoftext|>  . Sasural Simar Ka genres drama television series  <|endoftext|>  . Sasural Simar Ka genres drama television series  <|endoftext|>  . Sasural Simar Ka genres drama television series  <|endoftext|>  .']
To clarify, are you referring to the drama television series Sasural Simar Ka that has multiple genres?" [' Sasural Simar Ka genres drama television series  <|endoftext|>  . Sasural Simar Ka genres drama television series  <|endoftext|>  . Sasural Simar Ka genres drama television series  <|endoftext|>  .']
In my opinion, Sasural Simar Ka is a bad television series, and falls short

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from sacrebleu.metrics import BLEU
from sacrebleu import sentence_bleu,corpus_bleu

print ('Loading file...')
df = pd.read_csv('/content/drive/MyDrive/AA NLP244 Project Experiments/Evaluation/ChatGPT/chatgpt_eval.csv')
df = df.dropna(subset = ['text'])
df.head()

# df['SACC Perfect'] = df.apply(lambda x: (x['SACC']==100),axis=1)
# df['DA Perfect'] = df.apply(lambda x: x['DA Perfect']*(x['SACC']==100),axis=1)
# df['bleurt_max'] = df[['bleurt_ref1','bleurt_ref2','bleurt_ref3']].max(axis=1)

# df['bert_max'] = df[['bertsc_ref1','bertsc_ref2','bertsc_ref3']].max(axis=1)

df['LMProb'] = df['PPL'].apply(lambda x:round(1/np.log(x),2))

df['prob_match'] = df.apply(lambda x: 1*(x['da']==x['da']),axis=1)
df['var1'] = df.apply(lambda x:x['PPL']*x['prob_match']*x['LM_prob']/100,axis=1)
df['var3'] = df.apply(lambda x:x['pseudo_bleu']*x['prob_match']*x['LM_prob']/100,axis=1)
df['var2'] = df.apply(lambda x:x['PPL']*x['prob_match']*x['LM_prob']*x['pseudo_bleu']/100,axis=1)

rankers = ['var3','var1','var2','pseudo_bleu']

print ('mean_ps','cbleu','mean_bl', 'mean_be', 'mean_sacc', 'perf_sacc', 'perf_dasacc', 'perf_da')

for column in rankers:
    selected = []
        
    print (f'\n----------- RANKER: {column}--------------')

    for i, group in df.groupby('pseudo'):
        group = group[group[column]==group[column].max()]
        selected.append(group.index[0])
        # break


    # assert len(df_criteria)==360

    df_sel = df.loc[selected]

    df_sel.to_csv(f'/content/drive/MyDrive/AA NLP244 Project Experiments/Evaluation/ChatGPT/Ranked/chatgpt_ranked_{column}.csv',index=True)

    mean_ps = round(df_sel['pseudo_bleu'].mean(),4)
    # print (f'Pseudo bleu average:{mean_ps}')

    refs = df_sel['pseudo'].values.tolist()
    pred  = df_sel['text'].tolist()
    cbleu = corpus_bleu(pred, refs).score
    # print (f'Corpus_bleu: {cbleu}')

    # mean_bl = round(df_sel['bleurt_max'].mean(),4)*100
    # # print (f'BLEURT average of max:{mean_bl}')

    # mean_be = round(df_sel['bert_max'].mean(),4)*100
    # # print (f'BERT Score average of max:{mean_be}')

    #mean_sacc = round(df_sel['SACC'].mean(),4)
    # print (f'SACC Score average:{mean_sacc}')

    # perf_sacc = sum(df_sel['SACC Perfect'])/len(df_sel)*100
    # # print ('perfect SACC percent: ',perf_sacc)

    # perf_dasacc = sum((df_sel['DA Perfect']==1)&(df_sel['SACC Perfect']==1))/len(df_sel)*100
    # # print ('perfect SACC+DA percent: ',perf_dasacc)

    # perf_da = sum(df_sel['DA Perfect'])/len(df_sel)*100
    # # print ('perfect DA percent: ',perf_da)
    
    print ([round(i,2) for i in [mean_ps,cbleu]])

    #TV Stats 
# ----------- RANKER: var3--------------
# [10.28, 72.89, 93.9]

# ----------- RANKER: var1--------------
# [12.56, 72.55, 97.42]

# ----------- RANKER: var2--------------
# [10.4, 72.55, 96.85]

# ----------- RANKER: bbleu--------------
# [14.44, 60.51, 97.86]


# Laptop 
"""----------- RANKER: var3--------------
[10.48, 45.03, 80.33]

----------- RANKER: var1--------------
[10.86, 61.24, 86.65]

----------- RANKER: var2--------------
[14.51, 59.39, 89.43]

----------- RANKER: bbleu--------------
[18.43, 36.32, 90.86]"""

Loading file...
mean_ps cbleu mean_bl mean_be mean_sacc perf_sacc perf_dasacc perf_da

----------- RANKER: var3--------------
[6.32, 0.07]

----------- RANKER: var1--------------
[6.32, 0.07]

----------- RANKER: var2--------------
[6.62, 0.07]

----------- RANKER: pseudo_bleu--------------
[6.62, 0.07]


'----------- RANKER: var3--------------\n[10.48, 45.03, 80.33]\n\n----------- RANKER: var1--------------\n[10.86, 61.24, 86.65]\n\n----------- RANKER: var2--------------\n[14.51, 59.39, 89.43]\n\n----------- RANKER: bbleu--------------\n[18.43, 36.32, 90.86]'

In [ ]:
def tv_eval(filepath, named=True, model='bleurt-base-128'):
    def best_score(s1, s2, s3):
        return max(s1, s2, s3)

    df = pd.read_csv(filepath)
    #df_ref = pd.read_csv('/content/drive/MyDrive/AA NLP244 Project Experiments/Evaluation/ChatGPT/NLP244_TV_chatgpt_90_responses.csv')
    generated = list(df["text"])

    """ref0 = list(df_ref["0"])
    ref1 = list(df_ref["1"])
    ref2= list(df_ref["2"])"""
    #if named == False:
    df_mr = pd.read_csv('/content/drive/MyDrive/AA NLP244 Project Experiments/Evaluation/ChatGPT/NLP244_TV_chatgpt_90_responses.csv')
    mr = list(df_mr["mr"])
    """ref0 = remove_name(mr, ref0)
    ref1 = remove_name(mr, ref1)
    ref2 = remove_name(mr, ref2)"""

    metric = load_metric('bleurt', model)

    metric.add_batch(predictions=generated, references=mr)
    score0 = metric.compute()
    df0 = pd.DataFrame.from_dict(score0)

    """metric.add_batch(predictions=generated, references=ref1)
    score1 = metric.compute()
    df1 = pd.DataFrame.from_dict(score1)

    metric.add_batch(predictions=generated, references=ref2)
    score2 = metric.compute()
    df2 = pd.DataFrame.from_dict(score2)"""

    result = pd.concat([df0, df0, df0], axis=1)
    result.columns = ['score0', 'score1', 'score2']
    result['best_score'] = result.apply(
        lambda x: best_score(x.score0, x.score1, x.score2), axis=1)
    
    average_score = sum(result['best_score'])/len(result['best_score'])
    print(f'Average_score: {round(average_score, 3)}')
    return result
    
    average_score = sum(result['best_score'])/len(result['best_score'])
    print(f'Average_score: {round(average_score, 3)}')
    return result

In [ ]:
ex_n = 'ex44'
#dir = f'/content/drive/MyDrive/KG-NLG Jurassic/personage_outputs/{ex_n}'
filepath = '/content/drive/MyDrive/AA NLP244 Project Experiments/Evaluation/ChatGPT/NLP244_TV_chatgpt_90_responses.csv'

result = tv_eval(filepath, named=True, model='bleurt-base-128')
result.to_csv( '/content/drive/MyDrive/AA NLP244 Project Experiments/Evaluation/ChatGPT/chatgpt_custom_output_scores.csv', index=None)
    

## Jurassic

In [ ]:
start_time = time.time()
perplexity = load("perplexity", module_type="metric")

#USE THIS TO GET SCORES 
 #Perplexity 
 #LM_prob 
 #Pseudo BLEU
 # 
def get_scores_optimized(input_file, output_file): 
  df = pd.read_csv(input_file)
  gens = df["text"].fillna(' ')
  pseudo = df["pseudo"].fillna(' ')

  results = perplexity.compute(model_id='gpt2',
                             predictions=gens )
  df['LM_prob'] = [1/i for i in results['perplexities']]
  df['PPL'] = results['perplexities']
  
  for i, g in enumerate(gens):
    print(g, [pseudo[i]])
    sentence_bleu(g, [pseudo[i]]).score
  
  df['pseudo_bleu'] = [ sentence_bleu(g, [pseudo[i]]).score for i, g in enumerate(gens)]
 
  df.to_csv(output_file)


# print("10")
# get_scores_optimized("/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/tst/10_one/filtered_bleurt/personality-vertical-tst-10-filter.csv", "/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/tst/10_one/filtered_bleurt/personality-vertical-tst-10-filter-lm.csv")
# print("10 not filtered")
# get_scores_optimized("/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/tst/10_one/personality-vertical-tst-10.csv", "/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/tst/10_one/personality-vertical-tst-10-lm.csv")
# print(36)
# get_scores_optimized("/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/tst/36_one/personality-vertical-tst-36.csv", "/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/tst/36_one/personality-vertical-tst-36-lm.csv")
# print(1)
# get_scores_optimized("/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/tst/1_all/personality-vertical-tst-1.csv", "/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/tst/1_all/personality-vertical-tst-1-lm.csv")
#print(6)
#get_scores_optimized("/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/viggo-experiments/viggo-vertical-filterd-10.csv", "/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/viggo-experiments/viggo-vertical-filterd-10-lmbleu.csv")
get_scores_optimized("/content/drive/MyDrive/AA NLP244 Project Experiments/Evaluation/Jurassic/Jurassic_Outputs.csv", "/content/drive/MyDrive/AA NLP244 Project Experiments/Evaluation/Jurassic/jurassic_eval.csv")
print("--- %s seconds ---" % (time.time() - start_time))

Using pad_token, but it is not set yet.


  0%|          | 0/63 [00:00<?, ?it/s]

Spaced is a sitcom. Have you ever watched it? ['Spaced  genres  sitcom  Spaced  genres  sitcom  ']
One of my favorite British sitcoms is Spaced. Have you watched it? ['Spaced  genres  sitcom  Spaced  genres  sitcom  ']
You've probably heard of Spaced, but did you know it's a sitcom, too? ['Spaced  genres  sitcom  Spaced  genres  sitcom  ']
If you're looking for a sitcom with science fiction elements, I can recommend this one. It's called Spaced. ['Spaced  genres  sitcom  Spaced  genres  sitcom  ']
If you're looking for a new sitcom to try, you can try Spaced. It's available on Netflix. ['Spaced  genres  sitcom  Spaced  genres  sitcom  ']
We're talking about sitcoms, right? Have you watched this British sitcom called Spaced before? ['Spaced  genres  sitcom  Spaced  genres  sitcom  ']
If you're looking for a sitcom game, Spaced is one you might want to try. It came out in 1999 and was created by Simon Pegg and Nick Frost. ['Spaced  genres  sitcom  Spaced  genres  sitcom  ']
I was watchin

In [ ]:
print ('Loading file...')
df = pd.read_csv('/content/drive/MyDrive/AA NLP244 Project Experiments/Evaluation/Jurassic/jurassic_eval.csv')
df = df.dropna(subset = ['text'])
df.head()

# df['SACC Perfect'] = df.apply(lambda x: (x['SACC']==100),axis=1)
# df['DA Perfect'] = df.apply(lambda x: x['DA Perfect']*(x['SACC']==100),axis=1)
# df['bleurt_max'] = df[['bleurt_ref1','bleurt_ref2','bleurt_ref3']].max(axis=1)

# df['bert_max'] = df[['bertsc_ref1','bertsc_ref2','bertsc_ref3']].max(axis=1)

df['LMProb'] = df['PPL'].apply(lambda x:round(1/np.log(x),2))

df['prob_match'] = df.apply(lambda x: 1*(x['da']==x['da']),axis=1)
df['var1'] = df.apply(lambda x:x['PPL']*x['prob_match']*x['LM_prob']/100,axis=1)
df['var3'] = df.apply(lambda x:x['pseudo_bleu']*x['prob_match']*x['LM_prob']/100,axis=1)
df['var2'] = df.apply(lambda x:x['PPL']*x['prob_match']*x['LM_prob']*x['pseudo_bleu']/100,axis=1)

rankers = ['var3','var1','var2','pseudo_bleu']

print ('mean_ps','cbleu','mean_bl', 'mean_be', 'mean_sacc', 'perf_sacc', 'perf_dasacc', 'perf_da')

for column in rankers:
    selected = []
        
    print (f'\n----------- RANKER: {column}--------------')

    for i, group in df.groupby('pseudo'):
        group = group[group[column]==group[column].max()]
        selected.append(group.index[0])
        # break


    # assert len(df_criteria)==360

    df_sel = df.loc[selected]

    df_sel.to_csv(f'/content/drive/MyDrive/AA NLP244 Project Experiments/Evaluation/Jurassic/Ranked/jurassic_ranked_{column}.csv',index=True)

    mean_ps = round(df_sel['pseudo_bleu'].mean(),4)
    # print (f'Pseudo bleu average:{mean_ps}')

    refs = df_sel['pseudo'].values.tolist()
    pred  = df_sel['text'].tolist()
    cbleu = corpus_bleu(pred, refs).score
    # print (f'Corpus_bleu: {cbleu}')

    # mean_bl = round(df_sel['bleurt_max'].mean(),4)*100
    # # print (f'BLEURT average of max:{mean_bl}')

    # mean_be = round(df_sel['bert_max'].mean(),4)*100
    # # print (f'BERT Score average of max:{mean_be}')

    #mean_sacc = round(df_sel['SACC'].mean(),4)
    # print (f'SACC Score average:{mean_sacc}')

    # perf_sacc = sum(df_sel['SACC Perfect'])/len(df_sel)*100
    # # print ('perfect SACC percent: ',perf_sacc)

    # perf_dasacc = sum((df_sel['DA Perfect']==1)&(df_sel['SACC Perfect']==1))/len(df_sel)*100
    # # print ('perfect SACC+DA percent: ',perf_dasacc)

    # perf_da = sum(df_sel['DA Perfect'])/len(df_sel)*100
    # # print ('perfect DA percent: ',perf_da)
    
    print ([round(i,2) for i in [mean_ps,cbleu]])

    #TV Stats 
# ----------- RANKER: var3--------------
# [10.28, 72.89, 93.9]

# ----------- RANKER: var1--------------
# [12.56, 72.55, 97.42]

# ----------- RANKER: var2--------------
# [10.4, 72.55, 96.85]

# ----------- RANKER: bbleu--------------
# [14.44, 60.51, 97.86]


# Laptop 
"""----------- RANKER: var3--------------
[10.48, 45.03, 80.33]

----------- RANKER: var1--------------
[10.86, 61.24, 86.65]

----------- RANKER: var2--------------
[14.51, 59.39, 89.43]

----------- RANKER: bbleu--------------
[18.43, 36.32, 90.86]"""

Loading file...
mean_ps cbleu mean_bl mean_be mean_sacc perf_sacc perf_dasacc perf_da

----------- RANKER: var3--------------
[26.12, 0.07]

----------- RANKER: var1--------------
[14.43, 0.1]

----------- RANKER: var2--------------
[40.98, 0.08]

----------- RANKER: pseudo_bleu--------------
[40.98, 0.08]


'----------- RANKER: var3--------------\n[10.48, 45.03, 80.33]\n\n----------- RANKER: var1--------------\n[10.86, 61.24, 86.65]\n\n----------- RANKER: var2--------------\n[14.51, 59.39, 89.43]\n\n----------- RANKER: bbleu--------------\n[18.43, 36.32, 90.86]'

## GPT Neo

In [ ]:
start_time = time.time()
perplexity = load("perplexity", module_type="metric")

#USE THIS TO GET SCORES 
 #Perplexity 
 #LM_prob 
 #Pseudo BLEU
 # 
def get_scores_optimized(input_file, output_file): 
  df = pd.read_csv(input_file)
  gens = df["text"].fillna(' ')
  pseudo = df["pseudo"].fillna(' ')

  results = perplexity.compute(model_id='gpt2',
                             predictions=gens )
  df['LM_prob'] = [1/i for i in results['perplexities']]
  df['PPL'] = results['perplexities']
  
  for i, g in enumerate(gens):
    print(g, [pseudo[i]])
    sentence_bleu(g, [pseudo[i]]).score
  
  df['pseudo_bleu'] = [ sentence_bleu(g, [pseudo[i]]).score for i, g in enumerate(gens)]
 
  df.to_csv(output_file)


# print("10")
# get_scores_optimized("/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/tst/10_one/filtered_bleurt/personality-vertical-tst-10-filter.csv", "/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/tst/10_one/filtered_bleurt/personality-vertical-tst-10-filter-lm.csv")
# print("10 not filtered")
# get_scores_optimized("/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/tst/10_one/personality-vertical-tst-10.csv", "/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/tst/10_one/personality-vertical-tst-10-lm.csv")
# print(36)
# get_scores_optimized("/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/tst/36_one/personality-vertical-tst-36.csv", "/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/tst/36_one/personality-vertical-tst-36-lm.csv")
# print(1)
# get_scores_optimized("/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/tst/1_all/personality-vertical-tst-1.csv", "/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/tst/1_all/personality-vertical-tst-1-lm.csv")
#print(6)
#get_scores_optimized("/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/viggo-experiments/viggo-vertical-filterd-10.csv", "/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/viggo-experiments/viggo-vertical-filterd-10-lmbleu.csv")
get_scores_optimized("/content/drive/MyDrive/AA NLP244 Project Experiments/Evaluation/GPT Neo/GPTNeo_Gen.csv", "/content/drive/MyDrive/AA NLP244 Project Experiments/Evaluation/GPT Neo/GPTNEO_eval.csv")
print("--- %s seconds ---" % (time.time() - start_time))

Using pad_token, but it is not set yet.


  0%|          | 0/57 [00:00<?, ?it/s]

["GenerationResult(text='The\\n                                                              '"] [' Sasural Simar Ka genres drama television series  <|endoftext|>  . Sasural Simar Ka genres drama television series  <|endoftext|>  . Sasural Simar Ka genres drama television series  <|endoftext|>  . Sasural Simar Ka genres drama television series  <|endoftext|>  . Sasural Simar Ka genres drama television series  <|endoftext|>  . Sasural Simar Ka genres drama television series  <|endoftext|>  . Sasural Simar Ka genres drama television series  <|endoftext|>  . Sasural Simar Ka genres drama television series  <|endoftext|>  .']
["GenerationResult(text='The\\n                                                              '"] [' Sasural Simar Ka genres drama television series  <|endoftext|>  . Sasural Simar Ka genres drama television series  <|endoftext|>  . Sasural Simar Ka genres drama television series  <|endoftext|>  . Sasural Simar Ka genres drama television series  <|endoftext|>  . Sasura

In [ ]:
print ('Loading file...')
df = pd.read_csv('/content/drive/MyDrive/AA NLP244 Project Experiments/Evaluation/GPT Neo/GPTNEO_eval.csv')
df = df.dropna(subset = ['text'])
df.head()

# df['SACC Perfect'] = df.apply(lambda x: (x['SACC']==100),axis=1)
# df['DA Perfect'] = df.apply(lambda x: x['DA Perfect']*(x['SACC']==100),axis=1)
# df['bleurt_max'] = df[['bleurt_ref1','bleurt_ref2','bleurt_ref3']].max(axis=1)

# df['bert_max'] = df[['bertsc_ref1','bertsc_ref2','bertsc_ref3']].max(axis=1)

df['LMProb'] = df['PPL'].apply(lambda x:round(1/np.log(x),2))

df['prob_match'] = df.apply(lambda x: 1*(x['da']==x['da']),axis=1)
df['var1'] = df.apply(lambda x:x['PPL']*x['prob_match']*x['LM_prob']/100,axis=1)
df['var3'] = df.apply(lambda x:x['pseudo_bleu']*x['prob_match']*x['LM_prob']/100,axis=1)
df['var2'] = df.apply(lambda x:x['PPL']*x['prob_match']*x['LM_prob']*x['pseudo_bleu']/100,axis=1)

rankers = ['var3','var1','var2','pseudo_bleu']

print ('mean_ps','cbleu','mean_bl', 'mean_be', 'mean_sacc', 'perf_sacc', 'perf_dasacc', 'perf_da')

for column in rankers:
    selected = []
        
    print (f'\n----------- RANKER: {column}--------------')

    for i, group in df.groupby('pseudo'):
        group = group[group[column]==group[column].max()]
        selected.append(group.index[0])
        # break


    # assert len(df_criteria)==360

    df_sel = df.loc[selected]

    df_sel.to_csv(f'/content/drive/MyDrive/AA NLP244 Project Experiments/Evaluation/GPT Neo/Ranked/GPTNeo_ranked_{column}.csv',index=True)

    mean_ps = round(df_sel['pseudo_bleu'].mean(),4)
    # print (f'Pseudo bleu average:{mean_ps}')

    refs = df_sel['pseudo'].values.tolist()
    pred  = df_sel['text'].tolist()
    cbleu = corpus_bleu(pred, refs).score
    # print (f'Corpus_bleu: {cbleu}')

    # mean_bl = round(df_sel['bleurt_max'].mean(),4)*100
    # # print (f'BLEURT average of max:{mean_bl}')

    # mean_be = round(df_sel['bert_max'].mean(),4)*100
    # # print (f'BERT Score average of max:{mean_be}')

    #mean_sacc = round(df_sel['SACC'].mean(),4)
    # print (f'SACC Score average:{mean_sacc}')

    # perf_sacc = sum(df_sel['SACC Perfect'])/len(df_sel)*100
    # # print ('perfect SACC percent: ',perf_sacc)

    # perf_dasacc = sum((df_sel['DA Perfect']==1)&(df_sel['SACC Perfect']==1))/len(df_sel)*100
    # # print ('perfect SACC+DA percent: ',perf_dasacc)

    # perf_da = sum(df_sel['DA Perfect'])/len(df_sel)*100
    # # print ('perfect DA percent: ',perf_da)
    
    print ([round(i,2) for i in [mean_ps,cbleu]])

    #TV Stats 
# ----------- RANKER: var3--------------
# [10.28, 72.89, 93.9]

# ----------- RANKER: var1--------------
# [12.56, 72.55, 97.42]

# ----------- RANKER: var2--------------
# [10.4, 72.55, 96.85]

# ----------- RANKER: bbleu--------------
# [14.44, 60.51, 97.86]


# Laptop 
"""----------- RANKER: var3--------------
[10.48, 45.03, 80.33]

----------- RANKER: var1--------------
[10.86, 61.24, 86.65]

----------- RANKER: var2--------------
[14.51, 59.39, 89.43]

----------- RANKER: bbleu--------------
[18.43, 36.32, 90.86]"""

Loading file...
mean_ps cbleu mean_bl mean_be mean_sacc perf_sacc perf_dasacc perf_da

----------- RANKER: var3--------------
[0.14, 0.09]

----------- RANKER: var1--------------
[0.14, 0.09]

----------- RANKER: var2--------------
[0.15, 0.09]

----------- RANKER: pseudo_bleu--------------
[0.15, 0.09]


'----------- RANKER: var3--------------\n[10.48, 45.03, 80.33]\n\n----------- RANKER: var1--------------\n[10.86, 61.24, 86.65]\n\n----------- RANKER: var2--------------\n[14.51, 59.39, 89.43]\n\n----------- RANKER: bbleu--------------\n[18.43, 36.32, 90.86]'

## T5

In [ ]:
start_time = time.time()
perplexity = load("perplexity", module_type="metric")

#USE THIS TO GET SCORES 
 #Perplexity 
 #LM_prob 
 #Pseudo BLEU
 # 
def get_scores_optimized(input_file, output_file): 
  df = pd.read_csv(input_file)
  gens = df["text"].fillna(' ')
  pseudo = df["pseudo"].fillna(' ')

  results = perplexity.compute(model_id='gpt2',
                             predictions=gens )
  df['LM_prob'] = [1/i for i in results['perplexities']]
  df['PPL'] = results['perplexities']
  
  for i, g in enumerate(gens):
    print(g, [pseudo[i]])
    sentence_bleu(g, [pseudo[i]]).score
  
  df['pseudo_bleu'] = [ sentence_bleu(g, [pseudo[i]]).score for i, g in enumerate(gens)]
 
  df.to_csv(output_file)


# print("10")
# get_scores_optimized("/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/tst/10_one/filtered_bleurt/personality-vertical-tst-10-filter.csv", "/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/tst/10_one/filtered_bleurt/personality-vertical-tst-10-filter-lm.csv")
# print("10 not filtered")
# get_scores_optimized("/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/tst/10_one/personality-vertical-tst-10.csv", "/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/tst/10_one/personality-vertical-tst-10-lm.csv")
# print(36)
# get_scores_optimized("/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/tst/36_one/personality-vertical-tst-36.csv", "/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/tst/36_one/personality-vertical-tst-36-lm.csv")
# print(1)
# get_scores_optimized("/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/tst/1_all/personality-vertical-tst-1.csv", "/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/tst/1_all/personality-vertical-tst-1-lm.csv")
#print(6)
#get_scores_optimized("/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/viggo-experiments/viggo-vertical-filterd-10.csv", "/content/drive/MyDrive/KG-NLG Jurassic/Experiments IWSDS 2023/viggo-experiments/viggo-vertical-filterd-10-lmbleu.csv")
get_scores_optimized("/content/drive/MyDrive/AA NLP244 Project Experiments/Evaluation/T5/gpt_T5_V_1.5.csv", "/content/drive/MyDrive/AA NLP244 Project Experiments/Evaluation/T5/T5_eval.csv")
print("--- %s seconds ---" % (time.time() - start_time))

Using pad_token, but it is not set yet.


  0%|          | 0/6 [00:00<?, ?it/s]

learn how to rewrite dialogue act texts using the following examples. here is a [' Sasural Simar Ka genres drama television series  <|endoftext|>  . Sasural Simar Ka genres drama television series  <|endoftext|>  . Sasural Simar Ka genres drama television series  <|endoftext|>  . Sasural Simar Ka genres drama television series  <|endoftext|>  . Sasural Simar Ka genres drama television series  <|endoftext|>  . Sasural Simar Ka genres drama television series  <|endoftext|>  . Sasural Simar Ka genres drama television series  <|endoftext|>  . Sasural Simar Ka genres drama television series  <|endoftext|>  .']
learn how to rewrite dialogue act texts using the following examples. here is a [' Sasural Simar Ka genres drama television series  <|endoftext|>  . Sasural Simar Ka genres drama television series  <|endoftext|>  . Sasural Simar Ka genres drama television series  <|endoftext|>  .']
learn how to rewrite dialogue act texts using the following examples. here is a [' Sasural Simar Ka rati

In [ ]:
print ('Loading file...')
df = pd.read_csv('/content/drive/MyDrive/AA NLP244 Project Experiments/Evaluation/T5/T5_eval.csv')
df = df.dropna(subset = ['text'])
df.head()

# df['SACC Perfect'] = df.apply(lambda x: (x['SACC']==100),axis=1)
# df['DA Perfect'] = df.apply(lambda x: x['DA Perfect']*(x['SACC']==100),axis=1)
# df['bleurt_max'] = df[['bleurt_ref1','bleurt_ref2','bleurt_ref3']].max(axis=1)

# df['bert_max'] = df[['bertsc_ref1','bertsc_ref2','bertsc_ref3']].max(axis=1)

df['LMProb'] = df['PPL'].apply(lambda x:round(1/np.log(x),2))

df['prob_match'] = df.apply(lambda x: 1*(x['da']==x['da']),axis=1)
df['var1'] = df.apply(lambda x:x['PPL']*x['prob_match']*x['LM_prob']/100,axis=1)
df['var3'] = df.apply(lambda x:x['pseudo_bleu']*x['prob_match']*x['LM_prob']/100,axis=1)
df['var2'] = df.apply(lambda x:x['PPL']*x['prob_match']*x['LM_prob']*x['pseudo_bleu']/100,axis=1)

rankers = ['var3','var1','var2','pseudo_bleu']

print ('mean_ps','cbleu','mean_bl', 'mean_be', 'mean_sacc', 'perf_sacc', 'perf_dasacc', 'perf_da')

for column in rankers:
    selected = []
        
    print (f'\n----------- RANKER: {column}--------------')

    for i, group in df.groupby('pseudo'):
        group = group[group[column]==group[column].max()]
        selected.append(group.index[0])
        # break


    # assert len(df_criteria)==360

    df_sel = df.loc[selected]

    df_sel.to_csv(f'/content/drive/MyDrive/AA NLP244 Project Experiments/Evaluation/T5/Ranked/T5_ranked_{column}.csv',index=True)

    mean_ps = round(df_sel['pseudo_bleu'].mean(),4)
    # print (f'Pseudo bleu average:{mean_ps}')

    refs = df_sel['pseudo'].values.tolist()
    pred  = df_sel['text'].tolist()
    cbleu = corpus_bleu(pred, refs).score
    # print (f'Corpus_bleu: {cbleu}')

    # mean_bl = round(df_sel['bleurt_max'].mean(),4)*100
    # # print (f'BLEURT average of max:{mean_bl}')

    # mean_be = round(df_sel['bert_max'].mean(),4)*100
    # # print (f'BERT Score average of max:{mean_be}')

    #mean_sacc = round(df_sel['SACC'].mean(),4)
    # print (f'SACC Score average:{mean_sacc}')

    # perf_sacc = sum(df_sel['SACC Perfect'])/len(df_sel)*100
    # # print ('perfect SACC percent: ',perf_sacc)

    # perf_dasacc = sum((df_sel['DA Perfect']==1)&(df_sel['SACC Perfect']==1))/len(df_sel)*100
    # # print ('perfect SACC+DA percent: ',perf_dasacc)

    # perf_da = sum(df_sel['DA Perfect'])/len(df_sel)*100
    # # print ('perfect DA percent: ',perf_da)
    
    print ([round(i,2) for i in [mean_ps,cbleu]])

    #TV Stats 
# ----------- RANKER: var3--------------
# [10.28, 72.89, 93.9]

# ----------- RANKER: var1--------------
# [12.56, 72.55, 97.42]

# ----------- RANKER: var2--------------
# [10.4, 72.55, 96.85]

# ----------- RANKER: bbleu--------------
# [14.44, 60.51, 97.86]


# Laptop 
"""----------- RANKER: var3--------------
[10.48, 45.03, 80.33]

----------- RANKER: var1--------------
[10.86, 61.24, 86.65]

----------- RANKER: var2--------------
[14.51, 59.39, 89.43]

----------- RANKER: bbleu--------------
[18.43, 36.32, 90.86]"""

Loading file...
mean_ps cbleu mean_bl mean_be mean_sacc perf_sacc perf_dasacc perf_da

----------- RANKER: var3--------------
[2.96, 0.15]

----------- RANKER: var1--------------
[2.85, 0.14]

----------- RANKER: var2--------------
[3.33, 0.15]

----------- RANKER: pseudo_bleu--------------
[3.33, 0.15]


'----------- RANKER: var3--------------\n[10.48, 45.03, 80.33]\n\n----------- RANKER: var1--------------\n[10.86, 61.24, 86.65]\n\n----------- RANKER: var2--------------\n[14.51, 59.39, 89.43]\n\n----------- RANKER: bbleu--------------\n[18.43, 36.32, 90.86]'